In [1]:
import os, sys
sys.path.append(os.path.abspath('../utils'))
import pandas as pd
from sklearn.linear_model import LogisticRegression
import dca_fs_tools as dcat
import pickle
import numpy as np

# Backward stepwise selection based on mean net benefit

We implement a backward stepwise selection procedure based on net benefit.

Features are itteratively removed from the full model with the stopping rule that the model with maximum net benefit (or mean net benefit across all threshold probabilities).

This example is based on the scikit learn make_classification synthetic data set described in [00_synthetic_data_description.ipynb](./00_synthetic_data_description.ipynb).

Generate the synthetic data set:

In [2]:
n_sample = 1000

df_train, df_test, ind_var_names = dcat.make_class_dataset(n_sample = n_sample,
                                       n_features = 5,
                                       n_redundant = 0,
                                       random_state = 1001,
                                       n_informative = 4,
                                       n_clusters_per_class = 1,
                                       n_classes = 2)

Define the logistic regression model (this could be any type of model).

In [3]:
logreg = LogisticRegression(C=10**18)

Define a function for performing the backward stepwise selection procedure. The steps are described in code comments:

In [4]:
#from statkit.decision import net_benefit
#import numpy as np

#net_benefit(np.array([1,0,1]), np.array([0.1,0.2,0.3]), thresholds=np.array([0.5]))[1][0]

In [5]:
def backward_stepwise_dcafs(train_data, test_data, model, harms_dict, dependent="y", nb_threshold = "mnb" ):

    features_left = test_harms.keys()

    out = pd.DataFrame(columns = ["features", "nb"])

    # Initialize with full model

    model.fit(train_data[features_left], train_data[dependent])
    pred = model.predict_proba(test_data[features_left])

    if nb_threshold == "mnb":
        nb = dcat.mean_net_benefit(test_data[dependent], pred[:, 1], n_thresh=100)['mnb']
    else:
        nb = dcat.net_benefit(test_data[dependent], pred[:, 1] , thresholds = np.asarray([nb_threshold]))[1][0]


    harm = sum([test_harms[i] for i in features_left]) 
    nb = nb - harm
    out.loc[0] = pd.Series({"features": list(features_left), "nb": nb})

    for n_dropped in range(len(test_harms.keys())-1):
        #print(n_dropped)

        nb_per_drop = pd.DataFrame(columns = ["droped_feature", "model_features","nb"])
        for i, droped_feature in enumerate(features_left):
            #build a model with  the looped feature removed
    
            model_features = [i for i in features_left if not(i == droped_feature)]

            # build a model with the selected features
            model.fit(train_data[model_features], train_data[dependent])

            ## Make predictions on the test set:
            pred = model.predict_proba(test_data[model_features])

            # auc
            #auc = roc_auc_score(df_test[dependent],pred[:, 1])

            # mnb

            #nb = dcat.mean_net_benefit(test_data[dependent], pred[:, 1], n_thresh=100)['mnb']

            if nb_threshold == "mnb":
                nb = dcat.mean_net_benefit(test_data[dependent], pred[:, 1], n_thresh=100)['mnb']
            else:
                nb = dcat.net_benefit(test_data[dependent], pred[:, 1] , thresholds = np.asarray([nb_threshold]))[1][0]


            #Include test harms
            harm = sum([test_harms[i] for i in model_features]) 

            nb = nb - harm

            nb_per_drop.loc[i] = pd.Series({"droped_feature": droped_feature, "model_features": model_features, "nb": nb})


        excluded_feature = nb_per_drop[nb_per_drop['nb']==nb_per_drop['nb'].max()]
    
        ef = excluded_feature["droped_feature"].to_list()
        features_left = [f for f in features_left if not f in ef]
        mnb = excluded_feature["nb"].to_list()[0]
        out.loc[n_dropped+1] = pd.Series({"features": features_left, "nb": nb})



    return out

Run the selection procedure on the synthetic data:

In [6]:
test_harms = {"x0": 0.0, "x1": 0.015, "x2": 0.03, "x3": 0.045, "x4": 0.06 }

backward_selection = backward_stepwise_dcafs(df_train, df_test, logreg, test_harms, nb_threshold = "mnb")
backward_selection_pt_0 = backward_stepwise_dcafs(df_train, df_test, logreg, test_harms, nb_threshold = 0.8)
backward_selection_pt_1 = backward_stepwise_dcafs(df_train, df_test, logreg, test_harms, nb_threshold = 0.2)

# Save for later comparison to other methods

with open('../data/backward_selection.pkl', 'wb') as f:
    pickle.dump(backward_selection, f)

with open('../data/backward_selection_pt_0.pkl', 'wb') as f:
    pickle.dump(backward_selection_pt_0, f)

with open('../data/backward_selection_pt_1.pkl', 'wb') as f:
    pickle.dump(backward_selection_pt_1, f)

In [ ]:
# selected based on mean net benefit
backward_selection

,features,nb
0,"[x0, x1, x2, x3, x4]",0.142756
1,"[x0, x1, x2, x4]",0.124883
2,"[x0, x1, x4]",0.180078
3,"[x1, x4]",0.210887
4,[x1],0.209659


In [ ]:
# Selected based on a threshold probability of 0.8
backward_selection_pt_0

,features,nb
0,"[x0, x1, x2, x3, x4]",0.055
1,"[x0, x1, x3, x4]",-0.09
2,"[x0, x1, x4]",-0.065
3,"[x1, x4]",0.035
4,[x4],0.03


In [ ]:
# Selected based on a threshold probability of 0.2
backward_selection_pt_1

,features,nb
0,"[x0, x1, x2, x3, x4]",0.2575
1,"[x0, x1, x2, x4]",0.27875
2,"[x0, x1, x4]",0.315
3,"[x0, x1]",0.345
4,[x1],0.29375


### Conclusion

We see that the maximum net benefit occurs when features $\{ x1, x4\}$ are selected by mean net benefit, however when a specific probability threshold is used the results are sensitive to the value of the threshold.